In [ ]:
(
    300 * 300 * 72 * 4 ) / 1e6

In [ ]:
1512/72
## chunks to run a full time series. 
# a cluster of ten workers would load in about 2 seconds


# 

In [ ]:
(1405 * 621 ) / 300**2

In [ ]:
(24 * 23 * 4) / 1e6

In [ ]:
### 10-200 Mb range. in terms of total size  ; influenced by I/O latency and similar concerns.   (10 or 100ms latency for S3/dask scheduler). 
## size vs shape ; 
import xarray as xr
OPENDAP_url = 'https://cida.usgs.gov/thredds/dodsC/prism_v2'
ds_in = xr.open_dataset(OPENDAP_url, decode_times=False, chunks={'time': 72, 'lat': 312, 'lon':352})


In [ ]:
ds_in

In [ ]:
%%time
x = ds_in.isel(time=slice(0,72))#.to_zarr(mapper)

In [ ]:
ds_in.nbytes / 1e9

In [ ]:
import os
import logging
import configparser

logging.basicConfig(level=logging.INFO, force=True)

import xarray as xr
import dask
import fsspec
import rechunker
import zarr
os.environ['AWS_PROFILE'] = 'osn-renci'

awsconfig = configparser.ConfigParser()
awsconfig.read(
    os.path.expanduser('~/.aws/credentials') # default location... if yours is elsewhere, change this.
)
## NOTE:  The default will be for the OSN / RENCI profile and endpoint. Override this
## by setting environment variables before executing this cell/notebook.
_profile_nm  = os.environ.get('AWS_PROFILE', 'osn-renci')
_endpoint = os.environ.get('AWS_S3_ENDPOINT', 'https://renc.osn.xsede.org')
# Set environment vars based on parsed awsconfig
try:
    os.environ['AWS_ACCESS_KEY_ID']     = awsconfig[_profile_nm]['aws_access_key_id']
    os.environ['AWS_SECRET_ACCESS_KEY'] = awsconfig[_profile_nm]['aws_secret_access_key']
    os.environ['AWS_S3_ENDPOINT']       = _endpoint
    os.environ['AWS_PROFILE'] = _profile_nm
    os.environ['AWS_DEFAULT_PROFILE'] = _profile_nm
    os.environ['AWS_S3_REGION'] = _profile_nm
except KeyError:
    logging.error("Problem parsing the AWS credentials file. ")

In [ ]:
# INPUT: 
import os
import fsspec

OPENDAP_url = 'https://cida.usgs.gov/thredds/dodsC/prism_v2'

# OUTPUT:
FNAME = 'PRISM_v2_rsignell.zarr'


uname='gtrantham'
fsw = fsspec.filesystem('s3', 
    anon=False, 
    default_fill_cache=False, 
    skip_instance_cache=True, 
    client_kwargs={ 'endpoint_url': os.environ['AWS_S3_ENDPOINT'] },
)

workspace = f's3://rsignellbucket2/nhgf/{uname}/'

staging = fsw.get_mapper(workspace + 'staging_nebari.zarr')
outfile = fsw.get_mapper(workspace + FNAME)

for fname in ['staging_nebari.zarr', FNAME]:
    if fsw.exists(workspace + fname):
        logging.warning("Removing existing file/folder: %s", fname)
        fsw.rm(workspace + fname, recursive=True)

print("READY !!")

fsw.ls(workspace)


In [ ]:
from dask.distributed import progress, performance_report

try:
    from dask_gateway import Gateway
except ImportError:
    logging.error("Unable to import Dask Gateway.  Are you running in a cloud compute environment?\n")
    raise
os.environ['DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION'] = "1.0"

gateway = Gateway()
_options = gateway.cluster_options()
_options.conda_environment='users/users-pangeo'  ##<< this is the conda environment we use on nebari.
_options.profile = 'Medium Worker'
_env_to_add={}
aws_env_vars=['AWS_ACCESS_KEY_ID',
              'AWS_SECRET_ACCESS_KEY',
              'AWS_SESSION_TOKEN',
              'AWS_DEFAULT_REGION',
              'AWS_S3_ENDPOINT']
for _e in aws_env_vars:
    if _e in os.environ:
        _env_to_add[_e] = os.environ[_e]
_options.environment_vars = _env_to_add    
cluster = gateway.new_cluster(_options)          ##<< create cluster via the dask gateway
cluster.adapt(minimum=10, maximum=30)             ##<< Sets scaling parameters. 
client = cluster.get_client()

print("The 'cluster' object can be used to adjust cluster behavior.  i.e. 'cluster.adapt(minimum=10)'")
print("The 'client' object can be used to directly interact with the cluster.  i.e. 'client.submit(func)' ")
print(f"The link to view the client dashboard is:\n>  {client.dashboard_link}")

In [ ]:
%%time
#from dask.distributed import performance_report

with performance_report('progress.html'):
    ds_in.to_zarr(outfile, mode='w')


In [ ]:
test = xr.open_dataset(outfile, engine='zarr', chunks={})

In [ ]:
test


In [ ]:
156 * 2


In [ ]:
1405  / 352

strategy

size of chunk 
shape of chunk

calculate expected times for key read patterns (total extent; time-series)

In [ ]:
import hvplot.xarray

In [ ]:
%%time
da = test.tmx.sel(lon=-75, lat=41.1, method='nearest').load()


In [ ]:
da.plot()

In [ ]:
%%time
da = test.tmx.sel(time="1970-01").load()


In [ ]:
da.hvplot(x='lon', y='lat', rasterize=True, geo=True, tiles='OSM' )

In [ ]:
fsw.size(workspace + FNAME + "/ppt/3.1.1") / 1e6

In [ ]:
FNAME = 'PRISM_v2_rsignell.zarr'


uname='gtrantham'
fsw = fsspec.filesystem('s3', 
    anon=False, 
    default_fill_cache=False, 
    skip_instance_cache=True, 
    client_kwargs={ 'endpoint_url': os.environ['AWS_S3_ENDPOINT'] },
)


In [ ]:
fsw.ls(workspace + FNAME)

In [ ]:
client.close(); cluster.close()